# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet (`outputSet`) that lie on the convex hull.

In [25]:
# Reusable data structures and functions for the algorithms
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __repr__(self) -> str:
          return f"({self.x},{self.y})"

def distance(p, q):
    return abs(p.x - q.x) + abs(p.y - q.y)

def cross_product(p,q,r):
    return (q.x - p.x)*(r.y - p.y) - (r.x - p.x)*(q.y - p.y)


# Auxilliary functions for Jarvis March
def calculate_orientation(p, q, r):
        crossProduct = cross_product(p,q,r)

        if (crossProduct > 0): return 1         # Clockwise
        elif (crossProduct < 0): return 2       # Counterclockwise
        else: return 0                          # Collinear
        
def find_left_most_point(points):
        leftMostIndex = 0
        for i in range(1, len(points)):
                currentLeftMost = points[leftMostIndex]
                currentPoint = points[i]
                
                # Update to new left most point
                if currentPoint.x < currentLeftMost.x:
                        leftMostIndex = i
                
                # If multiple points have same x coord, prioritise largest y
                elif currentPoint.x == currentLeftMost.x:
                        if currentPoint.y > currentLeftMost.y:
                                leftMostIndex = i
                        
        return leftMostIndex

def jarvismarch(inputSet):      
    leftMostPoint = find_left_most_point(inputSet)
    outputSet = []
        
    origin = inputSet[leftMostPoint]
    outputSet.append(origin)
        
    p = leftMostPoint
        
    while True:
        q = (p + 1) % len(inputSet)
        for i in range(len(inputSet)):
                if i == p:
                        continue
                output = calculate_orientation(inputSet[p], inputSet[i], inputSet[q])
                if output == 0 and distance(inputSet[p], inputSet[i]) > distance(inputSet[p], inputSet[q]):
                        q = i
                        
                elif output == 2:
                        q = i
        p = q
        if p == leftMostPoint:
                break
                
        outputSet.append(inputSet[p])

    return outputSet



Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [26]:
# Auxilliary functions and data structure for Graham Scan
class PointSymbolTable: #technically a binary tree
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
    
    # in order traversal to list
    def valuesToList(self):
        return (self.left.valuesToList() if self.left else []) + [self.value] + (self.right.valuesToList() if self.right else [])
    
    # normal binary tree get
    def get(self, key): 
        if self.key == key:
            return self.value
        else:
            if self.key < key:
                return self.right.get(key)
            else:
                return self.left.get(key)

    # normal binary tree put
    def put(self, key, value, ref):
        if self.key == key:
            # always keep furthest point from reference point
            self.value = value if distance(value, ref) > distance(self.value, ref) else self.value
        elif self.key < key:
            if self.right is not None:
                self.right.put(key, value, ref)
            else:
                self.right = PointSymbolTable(key, value)
        else:
            if self.left is not None:
                self.left.put(key, value, ref)
            else:
                self.left = PointSymbolTable(key, value)
    
def cross_product(p1,p2,p3):
    return (p2.x - p1.x)*(p3.y - p1.y) - (p3.x - p1.x)*(p2.y - p1.y)

def sort_key(start,p1): #sort by -1/tan (angle)
    if p1.y == start.y:
        return float('inf') if p1.x - start.x < 0 else float('-inf')
    return - (p1.x - start.x) / (p1.y - start.y)



# Graham Scan
def grahamscan(inputSet):
    outputSet=[]
    #find point with lowest y co-ordinate - if y is the same take lowest x
    point0 = inputSet[0]
    for current in inputSet[1:]:
        if (current.y < point0.y) or ((current.y == point0.y) and (current.x < point0.x)):
            point0 = current
    #initialise symbol table with angle key and point value
    symTable = PointSymbolTable(sort_key(point0,inputSet[0]),inputSet[0])

    #calculate angles then sort by angle using symbol table
    for current in inputSet:
        if current != point0:
            symTable.put(sort_key(point0,current),current,point0)
    
    sortedPoints = [point0] + symTable.valuesToList()
    #push first two points onto stack
    outputSet.append(sortedPoints[0])
    outputSet.append(sortedPoints[1])
    for i in range(2,len(sortedPoints)):
        while len(outputSet) > 1 and cross_product(outputSet[-2],outputSet[-1],sortedPoints[i]) <= 0:
                #if < 0 then we are turning right so pop the last point from the stack - anticlockwise search
                outputSet.pop()
        outputSet.append(sortedPoints[i])
    return outputSet

Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [27]:
import matplotlib.pyplot as plt
import math
import random

class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __repr__(self) -> str:
          return f"({self.x},{self.y})"
      
      
def distance(p, q):
    return abs(p.x - q.x) + abs(p.y - q.y)

def is_better_orientation(p, q, r):
        crossProduct = (q.x - p.x) * (r.y - p.y) - (q.y - p.y) * (r.x - p.x)
        
        # Counterclockwise - accept
        if crossProduct < 0:
                return True
        # Collinear - only accept if further away
        elif crossProduct == 0:
                if distance(p, r) > distance(p, q):
                        return True
        
        return False

# used for copying over array elements
def subArrayCopy(arr, subArr, start, end):
    subArrayIndex = 0
    for i in range(start, end + 1):
        subArr[subArrayIndex] = arr[i]
        subArrayIndex += 1

def merge(arr, leftP, rightP):
    middleI = (rightP + leftP) // 2

    leftArray = [Point(0, 0)] * (middleI + 1 - leftP)
    rightArray = [Point(0, 0)] * (rightP - middleI)

    subArrayCopy(arr, leftArray, leftP, middleI)
    subArrayCopy(arr, rightArray, middleI + 1, rightP)

    leftArrayI = 0
    rightArrayI = 0
    arrPointer = leftP

    while leftArrayI < len(leftArray) and rightArrayI < len(rightArray):
        leftPoint = leftArray[leftArrayI]
        rightPoint = rightArray[rightArrayI]

        # orders inputSet by orientation, relative to leftmostpoint
        if is_better_orientation(arr[0], leftPoint, rightPoint):
            arr[arrPointer] = leftArray[leftArrayI]
            leftArrayI += 1
        else:
            arr[arrPointer] = rightArray[rightArrayI]
            rightArrayI += 1

        arrPointer += 1

    while leftArrayI < len(leftArray):
        arr[arrPointer] = leftArray[leftArrayI]
        leftArrayI += 1
        arrPointer += 1

    while rightArrayI < len(rightArray):
        arr[arrPointer] = rightArray[rightArrayI]
        rightArrayI += 1
        arrPointer += 1

def mergeSort(arr, leftP, rightP):
    if leftP < rightP:
        middleI = (rightP + leftP) // 2

        mergeSort(arr, leftP, middleI)
        mergeSort(arr, middleI + 1, rightP)

        merge(arr, leftP, rightP)

def graham_scan(inputSet):
    bottomLeftmostPoint = find_left_most_point(inputSet)
    
    #positions leftmostpoint as the first point
    #and sorts the rest of the points by orientation
    inputSet[bottomLeftmostPoint], inputSet[0] = inputSet[0], inputSet[bottomLeftmostPoint]
    mergeSort(inputSet, 1, len(inputSet) - 1)

    convexHull = []
    convexHull.append(inputSet[0])
    convexHull.append(inputSet[1])

    for i in range(2, len(inputSet)):

        # keeps popping inputSet off stack until back to anticlockwise convex hull
        while len(convexHull) > 1 and not is_better_orientation(convexHull[-2], convexHull[-1], inputSet[i]):
            convexHull.pop()
        convexHull.append(inputSet[i])

    return convexHull

def jarvisMarchModified(hulls):      
   
    #collects points across all hulls
    points = []
    for hull in hulls:
        points.extend(hull)
        
        
    #finds leftmost point
    leftMostPoint = find_left_most_point(points)
    
    outputSet = []
    origin = points[leftMostPoint]
    outputSet.append(origin)
        
        
    #follows normal jarvis, traversing and finding most anticlockwise point
    #updating q in the process, and each final q becomes the next p
    p = leftMostPoint
    
    while True:
        q = (p + 1) % len(points)
        for i in range(len(points)):
                if is_better_orientation(points[p], points[q], points[i]):
                        q = i
        p = q
        if p == leftMostPoint:
                break
                
        outputSet.append(points[p])

    return outputSet

def find_left_most_point(points):
        leftMostIndex = 0
        for i in range(1, len(points)):
                currentLeftMost = points[leftMostIndex]
                currentPoint = points[i]
                
                # Update to new left most point
                if currentPoint.x < currentLeftMost.x:
                        leftMostIndex = i
                
                # If multiple points have same x coord, prioritise largest y
                elif currentPoint.x == currentLeftMost.x:
                        if currentPoint.y > currentLeftMost.y:
                                leftMostIndex = i
                        
        return leftMostIndex

def createSubsets(inputSet):
    # size of convex hull approximated at root of total number of points
    m = int(math.sqrt(len(inputSet)))
    while len(inputSet) % m < 3 and len(inputSet) % m != 0:
        m +=1
    
    subsets = []
    
    for i in range(0, len(inputSet), m):
        subset = inputSet[i: i + m]
        subsets.append(subset)

    return subsets


def chen_scan(inputSet):
    # portions points into subsets
    subsets = createSubsets(inputSet)

    # uses grahams to form sub convex hulls
    hulls = [graham_scan(subset) for subset in subsets]

    # forms final convex hull using jarvis march
    outputSet = jarvisMarchModified(hulls)
    
    return outputSet


' Alternative Chens, faster only for really really large numbers of points \n\ndef createSubsets(inputSet, m):\n    subsets = []\n    for i in range(0, len(inputSet), m):\n        subset = inputSet[i: i + m]\n        if len(subset) < 3:\n            return []\n        subsets.append(subset)\n\n    return subsets\n\n# attempts to solve for a convex hull of size m\n# if it fails it then tries for a larger size m (previous m squared each failure)\ndef chensScan(inputSet):\n    t = 0\n    outputSet = []\n\n    while not outputSet:\n        t += 1\n        m = 2 ** 2 ** t\n\n    \n        # portions points into subsets\n        subsets = createSubsets(inputSet, m)\n\n        #invalid number of subsets, move onto next m\n        if not subsets:\n            continue\n        \n        # forms sub convex hulls using grahams\n        # then creates a final convex hull using jarvis\n        hulls = [grahamscan(subset) for subset in subsets]\n        outputSet = jarvisMarchModified(hulls, m)\n\n

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [56]:
import random
import math


class TestDataGenerator():
       
    #ADD YOUR CODE HERE

    '''
    We need
        > Colinear case proof
        > Average case analysis, varying n and h
        > Worst case analysis
        > Random case analysis

    '''
    def worst_case_points(self):

        # h = n for worst case
        # non collinear points, hence points are on the convex hull
        # in the form of a circle
        
        inputSet = set()
        i=0
        while True:
            inputSet.add(Point(int(math.cos(2*math.pi*i/self.n)), int(math.sin(2*math.pi*i/self.n))))
            if len(inputSet) == self.n:
                break
            i+=1
        return list(inputSet)

    def avg_case_points(self):
        inputSet = set()
        for i in range(self.h):
            inputSet.add(Point(random.randint(0, self.h), random.randint(0, self.h)))


    def __init__(self, n, h):
        self.n = n
        self.h = h
        self.avg_case_set = self.avg_case_points()
        self.worst_case_set = self.worst_case_points()
        self.best_case_set = self.best_case_points()
        self.colinear_points_set = self.colinear_points()

a = TestDataGenerator(10000, 6)
print(len(a.worst_case_set))
print(len(jarvismarch(a.worst_case_set)))



10000
4


Use the cell below to implement the requested **experimental framework** API.

In [5]:
import timeit
import matplotlib

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass

Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [6]:
# ADD YOUR TEST CODE HERE 

eval = ExperimentalFramework()

